In [6]:
import os
import shutil
from pathlib import Path

In [7]:
dataset_dir = Path('distracted-driver-detection')
train_dir = dataset_dir / 'train'
test_dir = dataset_dir / 'test'
binary_train_dir = dataset_dir / 'binary_train'
binary_test_dir = dataset_dir / 'binary_test'

In [8]:
(binary_train_dir / 'not_distracted').mkdir(parents=True, exist_ok=True)
(binary_train_dir / 'distracted').mkdir(parents=True, exist_ok=True)
(binary_test_dir / 'not_distracted').mkdir(parents=True, exist_ok=True)
(binary_test_dir / 'distracted').mkdir(parents=True, exist_ok=True)


In [9]:
def move_files(src_dir, dst_dir):
    if not src_dir.exists():
        print(f"Source directory {src_dir} does not exist.")
        return

    file_count = 0
    #having .jpg format in the for loop of files
    for file in src_dir.glob('*.jpg'):
        #copying ech file to dest dir through shutil func
        shutil.copy(file, dst_dir)
        file_count += 1

    print(f"Moved {file_count} files from {src_dir} to {dst_dir}")


In [10]:
move_files(train_dir / 'c0', binary_train_dir / 'not_distracted')
for class_dir in ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']:
    move_files(train_dir / class_dir, binary_train_dir / 'distracted')


Moved 2489 files from distracted-driver-detection\train\c0 to distracted-driver-detection\binary_train\not_distracted
Moved 2267 files from distracted-driver-detection\train\c1 to distracted-driver-detection\binary_train\distracted
Moved 2317 files from distracted-driver-detection\train\c2 to distracted-driver-detection\binary_train\distracted
Moved 2346 files from distracted-driver-detection\train\c3 to distracted-driver-detection\binary_train\distracted
Moved 2326 files from distracted-driver-detection\train\c4 to distracted-driver-detection\binary_train\distracted
Moved 2312 files from distracted-driver-detection\train\c5 to distracted-driver-detection\binary_train\distracted
Moved 2325 files from distracted-driver-detection\train\c6 to distracted-driver-detection\binary_train\distracted
Moved 2002 files from distracted-driver-detection\train\c7 to distracted-driver-detection\binary_train\distracted
Moved 1911 files from distracted-driver-detection\train\c8 to distracted-driver-dete

move_files(train_dir / 'c0', binary_test_dir / 'not_distracted')



In [11]:
move_files(test_dir , binary_test_dir / 'distracted')

OSError: [Errno 22] Invalid argument

In [12]:
print("Data preparation completed.")

Data preparation completed.


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [14]:
# Paths to data
train_data_dir = 'distracted-driver-detection/binary_train'
validation_data_dir = 'distracted-driver-detection/binary_test'

In [15]:
train_data_dir

'distracted-driver-detection/binary_train'

In [16]:
validation_data_dir

'distracted-driver-detection/binary_test'

In [17]:
# Image dimensions
img_width, img_height = 150, 150

In [18]:
# Data augmentation techiniques(shearing,rescaling,zooming,horizontal flip) and preprocessing
#: This rescales the pixel values of the images from a range of [0, 255] to [0, 1]. This is done by multiplying each pixel value by 1./255
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
print(train_datagen)
test_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
#generator will yield batches of 32 images at a time.

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), batch_size=32, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height), batch_size=32, class_mode='binary')


Found 22424 images belonging to 2 classes.
Found 82215 images belonging to 2 classes.


In [20]:
# Model building
#introduce non linearity in program i.e relu function
#32layer filters of 3*3 size
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),# reduces spatial dimensions by factor 2 helps in capturing dominant features
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),#2d to 1d
    Dense(512, activation='relu'),#512 neurons
    Dropout(0.5),#prevents overfitting of the data by setting 50% of input units to 0 during training
    Dense(1, activation='sigmoid')#output value 0 or 1 for final prediction
])

C:\Users\salag\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
# combines two techniques adaptive gradient algo and root mean square propagation ,
#it adjusts the learning rate this small rate provides stability during training
#measures the model whose output value is 0 or 1

In [22]:
history=model.fit(train_generator, steps_per_epoch=train_generator.samples // train_generator.batch_size, epochs=2, validation_data=validation_generator, validation_steps=validation_generator.samples // validation_generator.batch_size)


Epoch 1/2


C:\Users\salag\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


700/700 ━━━━━━━━━━━━━━━━━━━━ 0s 796ms/step - accuracy: 0.8840 - loss: 0.3276

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002A122C16610>
Traceback (most recent call last):

  File "C:\Users\salag\anaconda3\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\salag\anaconda3\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\salag\anaconda3\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\salag\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py", line 260, in _get_iterator
    for i, batch i

In [23]:
model.save('best_model.h5')
print("model savd as best_model.h5")
loss, accuracy = model.evaluate(validation_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

model savd as best_model.h5
 198/2570 ━━━━━━━━━━━━━━━━━━━━ 10:25 264ms/step - accuracy: 0.9677 - loss: 0.0944

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002A122FEFF60>
Traceback (most recent call last):

  File "C:\Users\salag\anaconda3\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\salag\anaconda3\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\salag\anaconda3\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\salag\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py", line 260, in _get_iterator
    for i, batch i

In [24]:

import cv2
import numpy as np
import tensorflow as tf

In [25]:
model = tf.keras.models.load_model('best_model.h5')

In [26]:
model

<Sequential name=sequential, built=True>

In [27]:
img_width, img_height = 150, 150

In [28]:
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)
    confidence_score=prediction[0][0]
    label='not_distracted' if confidence_score>0.5 else 'distracted'
    return label,confidence_score
    

In [29]:
image_path="C:\\Users\\salag\\OneDrive\\Desktop\\ml project\\distracted-driver-detection\\binary_test\\distracted\\img_3.jpg"

In [30]:
result,confidence=predict_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step


In [31]:
print(f'prediction:{result}')

prediction:distracted


In [32]:
from tensorflow.keras.models import load_model

In [21]:
pip install opencv-python-headless tensorflow

  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\salag\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('best_model.h5')

# Define image dimensions
img_width, img_height = 150, 150

def predict_frame(frame):
   

    img = cv2.resize(frame, (img_width, img_height))
    img = np.expand_dims(img, axis=0) / 255.0
    
    # Make prediction
    prediction = model.predict(img)
    confidence_score = prediction[0][0]
    
    # Interpret prediction
    label = 'distracted' if confidence_score < 0.5 else 'not_distracted'
    return label, confidence_score

def real_time_detection():
    cap = cv2.VideoCapture(0)  # Start video capture from webcam
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Predict distraction
        label, confidence = predict_frame(frame)
        
        # Display the label on the frame
        cv2.putText(frame, f'{label} ({confidence:.2f})', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if label == 'not_distracted' else (0, 0, 255), 2, cv2.LINE_AA)
        
        # Show the frame
        cv2.imshow('Driver Distraction Detection', frame)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

# Run the real-time detection
real_time_detection()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━